![HenryLogo](https://d31uz8lwfmyn8g.cloudfront.net/Assets/logo-henry-white-lg.png)

# **Proyecto integrador**

## **Planteamiento de la problemática**

Hemos sido contratados en el equipo de ciencia de datos en una consultora de renombre. Nos han asignado a un proyecto de estudio de atención en salud para un importante hospital. **Nuestro cliente desea saber las características más importantes que tienen los pacientes de cierto tipo de enfermedad que terminan en hospitalización.** Fue definido como caso aquel paciente que fue sometido a biopsia prostática y que en un periodo máximo de 30 días posteriores al procedimiento presentó fiebre, infección urinaria o sepsis; requiriendo manejo médico ambulatorio u hospitalizado para la resolución de la complicación y como control al paciente que fue sometido a biopsia prostática y que no presentó complicaciones infecciosas en el período de 30 días posteriores al procedimiento. Dado que tienen en su base de datos algunos datos referentes a los pacientes y resultados de exámenes diagnósticos, de pacientes hospitalizados y no hospitalizados, nos han entregado esta información.  

## Análisis exploratorio de datos de acuerdo con la descripción del dataset previo a la fase de modelamiento.

El estudio tiene como objetivo la clasificación de un paciente como hospitalizado o no hospitalizado.




In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import  train_test_split
from sklearn.metrics import r2_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.metrics import  classification_report


Carga de base de datos y revisen el tipo de cada columna del dataset.

In [ ]:
df = pd.read_excel(r"E:\Repositorios y bases de datos\Henry DS\Módulo 6\Proyecto_Integrador\Propuesta 1\BBDD_Hospitalización.xlsx")
df.head()

Comprobación de la correcta carga y análisis

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.isnull().sum()

In [ ]:
df.hist()

## Preparación de datos

Eliminación de datos irrelevantes

In [ ]:
# Aquí llevamos los valores atípicos a la mediana.
mediana = df['EDAD'].median()
df['EDAD'] = df['EDAD'].apply(lambda x: mediana if x > 100 else x)

# Outliers
ver_atipicos = df[df['EDAD']>100]
ver_atipicos

In [ ]:
# Eliminación de columnas irrelevantes
drop_columns = ['DIAS HOSPITALIZACION MQ', 'DIAS HOSPITALIZACIÓN UPC']
df = df.drop(drop_columns,axis=1)
df[df['EDAD']==75]

In [ ]:
# Eliminación de filas con datos vacios.
df = df.dropna()

# Normalización
df['NUMERO DE DIAS POST BIOPSIA EN QUE SE PRESENTA LA COMPLICACIÓN INFECCIOSA'].replace('NO', 0, inplace=True)
df['ENF. CRONICA PULMONAR OBSTRUCTIVA'].replace('SI, EPOC', 'SI',inplace=True)
df['ENF. CRONICA PULMONAR OBSTRUCTIVA'].replace('SI, ASMA', 'SI',inplace=True)

In [ ]:
# LLenar datos vacios en filas determinadas.
df['VOLUMEN PROSTATICO'].fillna('NO', inplace=True)

Verificaciones

In [ ]:
df.isnull().sum()

In [ ]:
df[(df['HOSPITALIZACION']!='NO')&( df['HOSPITALIZACION']!='SI')]
df[(df['ENF. CRONICA PULMONAR OBSTRUCTIVA'] != 'SI') & (df['ENF. CRONICA PULMONAR OBSTRUCTIVA'] != 'NO')]

Estandarización de datos

In [ ]:
columnas_binarias = ['DIABETES', 'HOSPITALIZACIÓN ULTIMO MES', 'BIOPSIAS PREVIAS', 
'VOLUMEN PROSTATICO', 'CUP', 'ENF. CRONICA PULMONAR OBSTRUCTIVA', 
'FIEBRE', 'ITU', 'HOSPITALIZACION']

for columna in columnas_binarias:
    df[columna] = df[columna].map({'SI':True, 'NO':False})

df.info()

Conversión a dummies

In [ ]:
columnas_dummies = ['ANTIBIOTICO UTILIAZADO EN LA PROFILAXIS', 
'TIPO DE CULTIVO', 'BIOPSIA', 'AGENTE AISLADO','PATRON DE RESISTENCIA']

df = pd.get_dummies(df, columns=columnas_dummies,
drop_first=True)

df

In [ ]:
#df.isnull().sum()

Matriz de correlación

In [ ]:
corr = df.corr()

# Diagaramado con matplotlib y Seaborn
plt.figure(figsize=(20,20))
sns.heatmap(corr, cbar = True,  square = True, annot=True, fmt= '.2f',annot_kws={'size': 15},
xticklabels= df.columns, 
yticklabels= df.columns,
cmap= 'coolwarm')
plt.show()

## Entrenamiento de modelo de regresión logística

In [17]:
# Selección
X = df.drop('HOSPITALIZACION', axis=1)
y = df['HOSPITALIZACION']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, random_state=1990)

In [23]:
# Balanceo con Smote
smt = SMOTE()

conteo_i1 = Counter(y_train)
conteo_i1


X_train, y_train = smt.fit_resample(X_train, y_train)
X_test, y_test = smt.fit_resample(X_test, y_test)


conteo_i2 = Counter(y_train)
conteo_i2


print(f'train antes {conteo_i1}')
print(f'train despues {conteo_i2}')

conteo train antes Counter({False: 416, True: 16})
conteo train despues Counter({False: 416, True: 416})


## Estandarización, entrenamiento y prueba

In [24]:
# Estandarización de los datos.
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

# Dividición y entrenamiento.
modelo = LogisticRegression()
modelo.fit(X_train, y_train)

# Prueba de predicción.
y_train_pred = modelo.predict(X_train)
y_test_pred = modelo.predict(X_test)

print(f'Resultado del la prediccion es: {accuracy_score(y_train, y_train_pred)}')
print(f'Resultado del la prediccion real es: {accuracy_score(y_test, y_test_pred)}')

Resultado del la prediccion es: 0.9975961538461539
Resultado del la prediccion real es: 0.9754901960784313


In [25]:
# Reporte de parametros
reporte = classification_report(y_test, y_test_pred)
print(reporte)

              precision    recall  f1-score   support

       False       0.98      0.97      0.98       102
        True       0.97      0.98      0.98       102

    accuracy                           0.98       204
   macro avg       0.98      0.98      0.98       204
weighted avg       0.98      0.98      0.98       204



In [26]:
# Matriz de confusion
matrix = confusion_matrix(y_test, y_test_pred)
matrix

array([[ 99,   3],
       [  2, 100]], dtype=int64)